In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
# 데이터 불러오기
df = pd.read_csv("./data/식물_전처리 - merged_df(3차).csv")



In [3]:
df.columns = df.columns.to_series().apply(lambda x: x.split("(")[0].strip())

In [4]:
# df.to_csv('./data/plant_3.csv',index=False)

In [5]:
df.head()

,cntntsSj,clCodeNm,열매여부,다육,로제트,덩굴,growthAraInfo,growthHgInfo,grwhTpCode,lighttdemanddoCodeNm,postngplaceCodeNm,managedemanddoCode,watercycleSummerCode
0,가울테리아,1,1,0,0,0,25,15,82002,2,4,58001,53003
1,개운죽,0,0,0,0,0,30,90,82002,1,2,58001,53002
2,골드크레스트 '윌마',0,0,0,0,0,50,100,82002,4,4,58002,53003
3,공작야자,0,0,0,0,0,100,300,82003,2,4,58003,53003
4,관엽베고니아,1,0,0,0,0,30,30,82002,2,4,58002,53003


### 코사인 유사도

In [6]:
# 유사도 계산을 위한 특성 벡터 추출
features = df.iloc[:, 1:]  # 식물 특성 열 선택
feature_vector = features.values  # NumPy 배열로 변환

# 코사인 유사도 계산
similarity_matrix = cosine_similarity(feature_vector)

In [7]:
features

,clCodeNm,열매여부,다육,로제트,덩굴,growthAraInfo,growthHgInfo,grwhTpCode,lighttdemanddoCodeNm,postngplaceCodeNm,managedemanddoCode,watercycleSummerCode
0,1,1,0,0,0,25,15,82002,2,4,58001,53003
1,0,0,0,0,0,30,90,82002,1,2,58001,53002
2,0,0,0,0,0,50,100,82002,4,4,58002,53003
3,0,0,0,0,0,100,300,82003,2,4,58003,53003
4,1,0,0,0,0,30,30,82002,2,4,58002,53003
...,...,...,...,...,...,...,...,...,...,...,...,...
212,0,0,0,0,0,40,150,82002,2,5,58003,53003
213,0,0,0,0,0,50,150,82002,4,7,58001,53003
214,1,0,0,0,0,30,30,82003,4,7,58001,53003
215,0,0,0,0,1,30,20,82003,2,4,58002,53003


In [18]:
# 사용자 입력에 따른 추천
user_preference_flower = True  # 사용자 입력에 따른 꽃 선호
user_watering_amount = 53003  # 사용자 입력에 따른 물주기 양
user_managed = 58001
user_grwhTpCode = 82002
user_growthAraInfo = 20
user_growthHgInfo = 20

In [21]:
feature_cols = ['clCodeNm', 'watercycleSummerCode', 'managedemanddoCode', 'grwhTpCode', 'growthAraInfo', 'growthHgInfo']
feature_vector = df[feature_cols].values


In [33]:
# 입력에 따른 유사한 식물 추천
user_vector = np.array([int(user_preference_flower), user_watering_amount, user_managed, user_grwhTpCode, user_growthAraInfo, user_growthHgInfo])
similarities = cosine_similarity(user_vector.reshape(1, -1),feature_vector )  # 입력 식물과의 유사도 벡터
# Find the indices of similar plants
similar_plant_indices = similarities.argsort()[0][::-1][1:]

# Get the recommended plant names
recommended_plants = df.loc[similar_plant_indices, 'cntntsSj'].tolist()

print("Recommended plants:", recommended_plants[0])

Recommended plants: 해마리아


In [34]:
# Best Match 계산
best_match_index = similar_plant_indices[0]  # 가장 유사한 식물 인덱스
best_match = similarities[0,best_match_index]  # 가장 유사한 식물과의 유사도

In [35]:
print("추천 식물:", recommended_plants[0])
print("Best Match:", best_match)

추천 식물: 해마리아
Best Match: 0.999999999906452


In [36]:
# 사용자 실제 선호 식물 정보
user_actual_preference = ['개운죽', '관엽베고니아', '구즈마니아']

# 추천 결과와 실제 선호 식물 정보 비교하여 True/False로 이진 레이블 생성
binary_recommended = [plant in recommended_plants for plant in user_actual_preference]

# Precision 계산
precision = sum(binary_recommended) / len(recommended_plants)

# Recall 계산
recall = sum(binary_recommended) / len(user_actual_preference)

# F1-score 계산
f1 = 2 * (precision * recall) / (precision + recall)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Precision: 0.013888888888888888
Recall: 1.0
F1-score: 0.0273972602739726


### 유클리디안 거리

In [37]:
def compute_euclidean_similarity(vector1, vector2):
    return np.linalg.norm(vector1 - vector2)

In [38]:
# 사용자 벡터 생성
user_vector = np.array([int(user_preference_flower), user_watering_amount, user_managed])

# 특성 벡터 추출
features = df.iloc[:, 1:].values
num_features = features.shape[1]

# 사용자 벡터의 길이 맞추기
user_vector = np.resize(user_vector, num_features)

# 유클리디안 유사도 계산
similarities_eu = np.array([compute_euclidean_similarity(user_vector, vector) for vector in features])
similar_plant_indices_eu = similarities_eu.argsort()[::-1][1:]  # 유사도가 높은 식물 인덱스 추출
recommended_plants_eu = df.loc[similar_plant_indices_eu, 'cntntsSj'].tolist()  # 추천 식물 이름 리스트

print("추천 식물:", recommended_plants_eu[0])

추천 식물: 해마리아
